<a href="https://colab.research.google.com/github/ttycp3/Classdesign/blob/main/NLEQuestion1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#G5119: Natural Language Engineering

##Computer Based Examination, 2023

Remember, you can add cells and change their type (between code and text/markdown) as required to answer the questions.

In [ ]:
# update your candidate number here
candidate_number = 625937

#Question 1 (50 Marks)

This question is about document similarity and information retrieval.

In [4]:
### do not change the code in this cell
# make sure you run this cell

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from math import log2, sqrt

nltk.download('stopwords')
nltk.download("punkt")
stop = stopwords.words('english')



# This is corpus of quotations, their sources and languages which we will use in this question.
corpus = [("All the world’s a stage, and all the men and women merely players.", "William Shakespeare", "English"),
          ("Ask not what your country can do for you; ask what you can do for your country.", "John Kennedy", "English"),
          ("Ask, and it shall be given you; seek, and you shall find.", "the Bible", "Greek"),
          ("Eighty percent of success is showing up.", "Woody Allen", "English"),
          ("Elementary, my dear Watson.", "Sherlock Holmes (character)", "English"),
          ("For those to whom much is given, much is required.", "the Bible", "Greek"),
          ("Frankly, my dear, I don't give a damn.", "Rhett Butler (character)", "English"),
          ("Genius is one percent inspiration and ninety-nine percent perspiration.", "Thomas Edison", "English")]

# This is a query that we will retrieve relevant documents for.
query = "Will I be given what I ask?"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


a) Preprocess the information in the list `corpus` by following the steps below.

i) Create three dictionaries - `quotes`, `sources` and `languages` - corresponding to the three elements in each tuple of `corpus`. Each key in these dictionaries should be the position in the original list and each value should be a string from the tuple at that position in the list.

Each value in `quotes` should be item 0 in a tuple, i.e. the quotation itself. Each value in `sources` should be item 1 in a tuple, i.e. the source associated with the quotation. Each value in `languages` should be item 2 in a tuple, i.e. the original language of the quotation.

So for example, the corpus `[("Hello", "Alice", "English")]` should be broken into three dictionaries: `{0: "Hello"}`, `{0: "Alice"}` and `{0: "English"}`.

(7 marks)

In [5]:
quotes = {}
sources = {}
languages = {}
for i, item in enumerate(corpus):
    quotes[i] = item[0]
    sources[i] = item[1]
    languages[i] = item[2]

ii) Tokenise the quotation strings in the dictionary `quotes` to produce a new dictionary with the same keys called `tokenised`, in which each value is a list of tokens.

So, for example the dictionary `{0: "Hello Bob"}` would become `{0: ["Hello", "Bob"]}`.

(4 marks)

In [6]:
tokenised = {}
for i in quotes:
    tokenised[i] = word_tokenize(quotes[i].lower())

iii) Case normalise the tokenised strings and remove stopwords and punctuation, putting the results into a new dictionary called `normalised`, with the same keys as `tokenised` and the values being normalised lists of tokens.

(6 marks)

In [7]:
normalised = {}
for i in tokenised:
    normalised[i] = [word for word in tokenised[i] if word.lower() not in stop and word.isalnum()]

iv) Describe two other forms of pre-processing that could be applied to text documents.

(4 marks)

1.Stemming/Lemmatization: Stemming is the process of reducing words to their base or root form, often by removing inflectional endings. For example, "running", "runs", and "runner" can be reduced to the stem "run". Lemmatization is a more sophisticated version that reduces words to their dictionary forms (lemmas), which are actual words instead of just stems. This helps in normalizing text data and grouping together words with similar meanings.

2.Part-of-Speech Tagging (POS) and Dependency Parsing: These advanced pre-processing techniques label each word with its grammatical role (POS tagging) or identify the syntactic relationships between words (dependency parsing). This can be useful for understanding the context and structure of sentences before feeding them into machine learning models. The NLTK library provides tools for these tasks too.

i) Calculate document frequencies for each token found in the documents contained in `normalised` and put the results in a dictionary called `df`. That is count how many different entries in the dictionary each token is found in.

(4 marks)

:b) Convert each document, ie each list of tokens, into a tfidf representation by following the steps below.

In [8]:
df = {}
for i, tokens in normalised.items():
    for token in set(tokens):
        df.setdefault(token, 0)
        df[token] += 1


ii) Calculate inverse document frequencies from the document frequencies derived in the previous question and put the results in a dictionary called `idf`.

$$IDF(w) = \log_2 \left( \frac{N}{DF(w)} \right) $$

where $N$ is the total number of documents and $DF(w)$ is the document frequency of the word $w$.

(4 marks)

In [9]:
N = len(normalised)
idf = {}
for token in df:
    idf[token] = log2(N / df[token])

iii) Convert each document in `normalised` from a list of tokens to a dictionary of term frequencies and put the results in a dictionary called `tf`.

The keys of `tf` should be positions in the original list `corpus` and the values should be the term frequency dictionaries, which map from tokens to frequency in each document.

So, for example, `{0: ["Hello", "Bob"]}` would become `{0: {"Hello": 1, "Bob": 1}}`.

(4 marks)

In [10]:
tf = {}
for i, tokens in normalised.items():
    tf[i] = {token: tokens.count(token) for token in set(tokens)}

iv) Convert the raw term frequencies in `tf` to tfidf values using the dictionary `idf`.

(3 marks)

In [11]:
tfidf = {}
for i, terms in tf.items():
    tfidf[i] = {token: tf[i][token] * idf[token] for token in terms}

c) In the following steps, preprocess the string `query` and convert it to a tfidf representation then use this to find relevant quotations in the index `tfidf`.

i) Define a function `dot` which takes two dictionaries containing tfidf values as inputs and calculates their dot product.

(3 marks)

In [12]:
def dot(d1, d2):
    sum = 0.0
    for token in d1:
        if token in d2:
            sum += d1[token] * d2[token]
    return sum


ii) Define a function, `sim`, which takes two dictionaries containing tfidf values as inputs and , using your `dot` function, calculates their cosine similarity.

(3 marks)

In [13]:
def sim(d1, d2):
    return dot(d1, d2) / sqrt(dot(d1, d1) * dot(d2, d2))


iii) Preprocess the string `query` and convert it to a tfidf representation. Then calculate its cosine similarity to all the documents in the dictionary `tfidf`.

For any document with a non-zero similarity, print out the similarity, source and language of the original quotation.

(8 marks)

In [14]:

query_tokens = word_tokenize(query.lower())
query_tokens = [word for word in query_tokens if word.lower() not in stop and word.isalnum()]
query_tf = {token: query_tokens.count(token) for token in set(query_tokens)}
query_tfidf = {token: query_tf[token] * idf[token] for token in query_tf}

similarities = []
for i, tfidf in tfidf.items():
    similarities.append(sim(query_tfidf, tfidf))

sorted_similarities = sorted(similarities, reverse=True)
for sim, i in zip(sorted_similarities, range(len(similarities))):
    if sim > 0:
        print("{}: {} ({})".format(sim, sources[i], languages[i]))


0.3922322702763681: William Shakespeare (English)
0.3592106040535498: John Kennedy (English)
0.20203050891044214: the Bible (Greek)
